# 0 - Import modules

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
import cimcb_lite as cb

In [2]:
import re
import os
pwd = os.getcwd()
print(pwd)

C:\Users\Admin\Desktop\Cours\INSA Lyon\S5-Génomique Médicale\multiomics-similarity-graph


In [3]:
# Tune the visual settings for figures in seaborn
sns.set_context(
    "notebook", font_scale=1.2, rc={"figure.figsize": (15,8), "axes.titlesize": 15}
)

from matplotlib import rcParams

rcParams["figure.figsize"] = 15, 8

sns.set_theme(style="darkgrid")

In [8]:
import sys
sys.path.append(os.path.join(pwd, 'bin'))

# 0 - Import data

In [5]:
peakTable_MTBLS17_init = pd.read_excel(os.path.join(pwd, 'data/MTBLS17/Peaklist_EXP1_POS.xlsx') , sheet_name='US_EXP1_POS_Annotated_Abbr')
print(f'Peak table shape : {peakTable_MTBLS17_init.shape}')
peakTable_MTBLS17_init.head()

Peak table shape : (1586, 388)


,mz,mono_mz,rt,FC,TT_pv,TT_qv,WX_pv,WX_qv,Exp1_CRR_1a_POS,Exp1_CRR_1b_POS,...,Exp1_HCC_59a_POS,Exp1_HCC_59b_POS,Exp1_HCC_60a_POS,Exp1_HCC_60b_POS,Exp1_HCC_63a_POS,Exp1_HCC_63b_POS,Exp1_HCC_64a_POS,Exp1_HCC_64b_POS,isotopes,adduct
0,69.034193,69.034193,245.927310,1.064983,0.361288,0.414002,0.418035,0.440384,230.127798,587.927618,...,713.427477,797.486999,944.502223,1008.902996,989.869316,1050.722511,1063.436804,1149.045224,NaN,NaN
1,69.034752,69.034752,139.172875,1.444227,0.016183,0.091260,0.072598,0.199058,118.194026,129.710850,...,90.783984,77.323598,33.814223,30.604424,42.869602,30.771758,389.046294,424.640566,NaN,NaN
2,69.070384,69.070384,33.994059,1.215627,0.007724,0.078745,0.000686,0.024863,757.591222,594.600666,...,298.010732,255.891087,282.365009,246.467121,171.359314,161.726294,323.123227,288.725470,NaN,NaN
3,69.070545,69.070545,373.031466,1.144033,0.407307,0.442863,0.234142,0.355057,107.927970,145.488720,...,220.801553,297.617438,606.661381,785.677459,392.755907,410.149582,427.651740,376.692099,NaN,NaN
4,70.065640,70.065640,22.760499,-1.026797,0.671203,0.543876,0.724250,0.565184,950.757846,896.401881,...,360.511751,356.275541,270.565775,277.458326,209.780709,236.145254,201.404361,210.853556,NaN,NaN


In [6]:
metadata = pd.read_csv(os.path.join(pwd,'data/MTBLS17/s_live_mtbls17.txt'), sep='\t')
print(f'Metadata shape : {metadata.shape}')
metadata.head()

Metadata shape : (1050, 18)


,Source Name,Characteristics[Organism],Term Source REF,Term Accession Number,Characteristics[Organism part],Term Source REF.1,Term Accession Number.1,Protocol REF,Sample Name,Factor Value[Experiment],Term Source REF.2,Term Accession Number.2,Factor Value[Disease],Term Source REF.3,Term Accession Number.3,Factor Value[Injection],Term Source REF.4,Term Accession Number.4
0,Human blood serum,Homo sapiens,NCBITAXON,http://purl.bioontology.org/ontology/NCBITAXON...,blood serum,BTO,http://purl.obolibrary.org/obo/BTO_0000133,Sample collection,Exp1_CRR_1a_NEG,Exp1,NaN,NaN,CRR,NaN,NaN,Inj1,NaN,NaN
1,Human blood serum,Homo sapiens,NCBITAXON,http://purl.bioontology.org/ontology/NCBITAXON...,blood serum,BTO,http://purl.obolibrary.org/obo/BTO_0000133,Sample collection,Exp1_CRR_1b_NEG,Exp1,NaN,NaN,CRR,NaN,NaN,Inj2,NaN,NaN
2,Human blood serum,Homo sapiens,NCBITAXON,http://purl.bioontology.org/ontology/NCBITAXON...,blood serum,BTO,http://purl.obolibrary.org/obo/BTO_0000133,Sample collection,Exp1_CRR_3a_NEG,Exp1,NaN,NaN,CRR,NaN,NaN,Inj1,NaN,NaN
3,Human blood serum,Homo sapiens,NCBITAXON,http://purl.bioontology.org/ontology/NCBITAXON...,blood serum,BTO,http://purl.obolibrary.org/obo/BTO_0000133,Sample collection,Exp1_CRR_3b_NEG,Exp1,NaN,NaN,CRR,NaN,NaN,Inj2,NaN,NaN
4,Human blood serum,Homo sapiens,NCBITAXON,http://purl.bioontology.org/ontology/NCBITAXON...,blood serum,BTO,http://purl.obolibrary.org/obo/BTO_0000133,Sample collection,Exp1_CRR_4a_NEG,Exp1,NaN,NaN,CRR,NaN,NaN,Inj1,NaN,NaN


### Below we will keep onky useful metadata

# 1 - Preprocess data

In [11]:
!pip install plotly

  Obtaining dependency information for plotly from https://files.pythonhosted.org/packages/a8/07/72953cf70e3bd3a24cbc3e743e6f8539abe6e3e6d83c3c0c83426eaffd39/plotly-5.18.0-py3-none-any.whl.metadata
  Obtaining dependency information for tenacity>=6.2.0 from https://files.pythonhosted.org/packages/f4/f1/990741d5bb2487d529d20a433210ffa136a367751e454214013b441c4575/tenacity-8.2.3-py3-none-any.whl.metadata
   ---------------------------------------- 15.6/15.6 MB 28.4 MB/s eta 0:00:00


Le chemin dâ€™accÃ¨s spÃ©cifiÃ© est introuvable.

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from explore_data_functions import *

## a - Transform data to adapt with input requirements

In [13]:
peakTable_MTBLS17 = peakTable_MTBLS17_init.copy()
peakTable_MTBLS17 = peakTable_MTBLS17.loc[:, 'Exp1_CRR_1a_POS':]
peakTable_MTBLS17 = peakTable_MTBLS17.drop(['isotopes', 'adduct'], axis=1)
peakTable_MTBLS17.head()

,Exp1_CRR_1a_POS,Exp1_CRR_1b_POS,Exp1_CRR_3a_POS,Exp1_CRR_3b_POS,Exp1_CRR_4a_POS,Exp1_CRR_4b_POS,Exp1_CRR_5a_POS,Exp1_CRR_5b_POS,Exp1_CRR_6a_POS,Exp1_CRR_6b_POS,...,Exp1_HCC_58a_POS,Exp1_HCC_58b_POS,Exp1_HCC_59a_POS,Exp1_HCC_59b_POS,Exp1_HCC_60a_POS,Exp1_HCC_60b_POS,Exp1_HCC_63a_POS,Exp1_HCC_63b_POS,Exp1_HCC_64a_POS,Exp1_HCC_64b_POS
0,230.127798,587.927618,300.579088,568.440320,237.746731,460.294192,440.768045,740.698257,612.552420,1133.597320,...,871.093823,1010.693479,713.427477,797.486999,944.502223,1008.902996,989.869316,1050.722511,1063.436804,1149.045224
1,118.194026,129.710850,212.674262,222.858966,124.304083,136.387626,138.485911,161.454543,257.832954,270.554464,...,29.157877,50.976520,90.783984,77.323598,33.814223,30.604424,42.869602,30.771758,389.046294,424.640566
2,757.591222,594.600666,658.022293,580.603252,645.234904,561.511359,461.754859,417.581034,731.752527,562.375061,...,174.268984,128.793853,298.010732,255.891087,282.365009,246.467121,171.359314,161.726294,323.123227,288.725470
3,107.927970,145.488720,98.527020,114.068949,47.450318,81.684234,75.641921,74.463995,112.473876,185.631654,...,311.326136,384.622123,220.801553,297.617438,606.661381,785.677459,392.755907,410.149582,427.651740,376.692099
4,950.757846,896.401881,646.353708,622.684905,731.368097,722.464761,1194.746673,1157.661719,843.330026,860.639631,...,300.492187,256.624484,360.511751,356.275541,270.565775,277.458326,209.780709,236.145254,201.404361,210.853556


In [15]:
peakTable_MTBLS17.insert(0, '', peakTable_MTBLS17_init['mz'].astype(str) + '@' + peakTable_MTBLS17_init['rt'].astype(str))
peakTable_MTBLS17 = peakTable_MTBLS17.transpose()
peakTable_MTBLS17.columns = peakTable_MTBLS17.iloc[0, :]
peakTable_MTBLS17 = peakTable_MTBLS17.drop(peakTable_MTBLS17.index[0], axis=0)
peakTable_MTBLS17 = peakTable_MTBLS17.reset_index()
peakTable_MTBLS17 = peakTable_MTBLS17.rename(columns={'index': 'Sample Name'})
peakTable_MTBLS17.index = range(peakTable_MTBLS17.shape[0])
peakTable_MTBLS17

,,69.034193421312@245.927309952246,69.0347519118582@139.172874889953,69.0703839837015@33.9940592171752,69.0705452827662@373.031465819143,70.0656402237219@22.7604988324661,71.085449630006@379.222418617272,71.0860210463123@415.343642215146,72.0810846503853@23.7091295964681,72.0806908589304@130.850159427121,...,261.189573361512@126.724785372488,262.254199507943@241.280596270886,262.254272674164@460.463648737246,262.254085874238@427.246028775549,263.103606879043@116.843782177653,263.128885223185@245.867508302895,263.249421103173@460.015626030012,264.132808031926@245.869666087669,264.269693587755@261.339524822674,264.268036067514@454.81158605591
0,Sample Name,Exp1_CRR_1a_POS,Exp1_CRR_1b_POS,Exp1_CRR_3a_POS,Exp1_CRR_3b_POS,Exp1_CRR_4a_POS,Exp1_CRR_4b_POS,Exp1_CRR_5a_POS,Exp1_CRR_5b_POS,Exp1_CRR_6a_POS,...,Exp1_HCC_58a_POS,Exp1_HCC_58b_POS,Exp1_HCC_59a_POS,Exp1_HCC_59b_POS,Exp1_HCC_60a_POS,Exp1_HCC_60b_POS,Exp1_HCC_63a_POS,Exp1_HCC_63b_POS,Exp1_HCC_64a_POS,Exp1_HCC_64b_POS
1,69.034193421312@245.927309952246,230.127798,587.927618,300.579088,568.44032,237.746731,460.294192,440.768045,740.698257,612.55242,...,871.093823,1010.693479,713.427477,797.486999,944.502223,1008.902996,989.869316,1050.722511,1063.436804,1149.045224
2,69.0347519118582@139.172874889953,118.194026,129.71085,212.674262,222.858966,124.304083,136.387626,138.485911,161.454543,257.832954,...,29.157877,50.97652,90.783984,77.323598,33.814223,30.604424,42.869602,30.771758,389.046294,424.640566
3,69.0703839837015@33.9940592171752,757.591222,594.600666,658.022293,580.603252,645.234904,561.511359,461.754859,417.581034,731.752527,...,174.268984,128.793853,298.010732,255.891087,282.365009,246.467121,171.359314,161.726294,323.123227,288.72547
4,69.0705452827662@373.031465819143,107.92797,145.48872,98.52702,114.068949,47.450318,81.684234,75.641921,74.463995,112.473876,...,311.326136,384.622123,220.801553,297.617438,606.661381,785.677459,392.755907,410.149582,427.65174,376.692099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1582,848.38745497379@245.903140535829,26.854367,442.007981,74.074524,375.213998,58.673974,359.679109,259.991447,663.288763,525.259255,...,599.674322,796.734808,429.124539,518.378844,571.473489,711.25957,871.67816,847.550776,643.123226,800.882347
1583,848.885621429239@245.902303453468,21.924561,402.357357,84.825422,360.015568,89.294382,344.620685,250.836823,606.269599,511.447837,...,630.116817,788.292939,420.993841,577.087196,603.145726,681.918379,852.241868,903.968504,663.230258,842.841985
1584,849.388345280949@245.887139929679,6.821398,281.384643,48.47349,234.676324,44.567909,183.387405,162.134148,404.381483,343.337359,...,367.375716,497.352725,271.368749,299.220871,351.708623,415.284426,532.747944,561.767012,399.842543,479.402034
1585,849.889131681996@245.885868301917,2.15699,128.797298,27.935645,82.68435,11.712702,69.262578,55.12298,180.196226,145.680151,...,186.036111,210.282461,102.24074,138.552479,161.286788,180.770813,223.508213,263.886891,143.443414,231.980527


In [16]:
metadata_subset = metadata[metadata['Sample Name'].isin(peakTable_MTBLS17['Sample Name'])]
print(f'Metadata shape : {metadata_subset.shape}')
metadata_subset.head()

KeyError: 'Sample Name'

In [ ]:
peakTable = pd.merge(left=metadata_subset[['Sample Name', 'Factor Value[Disease]']].rename(columns={'Factor Value[Disease]': 'Disease'}),
                     right=peakTable_MTBLS17,
                     on='Sample Name')
peakTable

In [ ]:
peakTable.to_csv(os.path.join(pwd,'data/MTBLS17/processed_peakTable.csv'), index=False, header=True)

### Separate intensities (in dataframe <code>X</code>) and metadata (in dataframe <code>metadata</code>)

In [ ]:
X = peakTable.iloc[:, 2:]
X

In [ ]:
metadata = peakTable.drop(X.columns, axis=1)
metadata

In [ ]:
target = metadata['Disease']


# 2 - Explore data

## a - Feature types

### For further analysis, we prefer peak table intensites as float

In [ ]:
peakTable[peakTable.columns[2:]] = peakTable[peakTable.columns[2:]].apply(pd.to_numeric, errors='coerce', axis=1)
peakTable

In [ ]:
plot_feature_types(peakTable)

### Re-subset X to have numerical values instead of object

In [ ]:
X = peakTable.iloc[:,2:]
X
#vers ligne 280 plein de 0 : surtout 276 et 277 dans les derniers métabolites

## b - Missing values exploration

In [ ]:
plot_infos_missing_values(X)
# checking NaN values

### The function <code>plot_infos_missing_values</code> was developed to spot values equal to <code>NaN</code> by default. Here there are no NaNs but missing values can be in another form (0, 1, a specific string e.g. *'Unknown'* or *'NA'*, ...). We can pass argument <code>na_values</code> to specify the type of missing values in the peak table.

In [ ]:
plot_infos_missing_values(X, na_values=0)

<div class="alert alert-block alert-info">
 Apply a threshold to discriminate patients that have more than 10% missing values.

In [ ]:
nb_missing = []
nb_patient = []
for i in range(len(X)) :
    nb_missing.append(plot_infos_missing_values(X.iloc[i], na_values=0, plot = False))
    if (nb_missing[i] > 1586/10) :
        nb_patient.append(i)
        print(metadata.iloc[i], "\n number of missing values : ", nb_missing[i], "\n")


<div class="alert alert-block alert-info">
 Generate new peak table without the patients with too many missing values
 and the associated X and metadata tables

In [ ]:
peakTable1 = peakTable.drop(nb_patient, axis=0)
peakTable1[270:280]

In [ ]:
X1 = peakTable1.iloc[:, 2:]
X1

In [ ]:
metadata1 = peakTable1.drop(X1.columns, axis=1)
metadata1


In [ ]:
target1 = metadata1['Disease']

## c - Correlation between features

In [ ]:
plot_correlation_matrix(X1, threshold=0.9)

## d - Remove strongly correlated features (to potentially avoid issues during further analysis)

<div class="alert alert-block alert-info">
We also tried to decorrelate the data

In [ ]:
X_remove_corr = remove_correlated_features(X1)
X_remove_corr

In [ ]:
peakTable_remove_corr = pd.concat([metadata1, X_remove_corr], axis=1)
peakTable_remove_corr

## e - PCA

In [ ]:
if plot_infos_missing_values(X, plot=False) == 0:
    cb.plot.pca(X1, pcx=1, pcy=2, group_label=target1)
else:
    print('NaNs in peak table, please impute before using PCA')

## f - Target visualisation

In [ ]:
plot_target(target1)

# 3 - Impute missing values

In [ ]:
from missing_value_imputation_functions import *

### We observed before that missing values are here 0's. For further analysis, we prefer to avoid having 0's for mathematical purposes, we can replace these zeros by other values

<div class="alert alert-block alert-info">
Decomment next cell to use decorrelated values 

In [ ]:
X1 = X_remove_corr

In [ ]:
plot_infos_missing_values(X1, na_values=0)

### Set 0 to NaN to then impute with chosen imputer

In [ ]:
X_with_Nan = X1.copy()
X_with_Nan[X_with_Nan == 0] = np.nan

### Path to save imputed peak tables


In [ ]:
path_peakTable_imputed = os.path.join(pwd, 'data/synthetic_dataset/imputed_peak_tables/')

if os.path.exists(path_peakTable_imputed):
    print(f'Directory {path_peakTable_imputed} already exists !')
else:
    os.makedirs(path_peakTable_imputed)
    print(f'Directory {path_peakTable_imputed} created !')

<div class="alert alert-block alert-info">
Kernel is crashing if we run choose_best_imputation_method for the whole dataset.
    
We run it on a subset of the dataset that has a lot of missing values, and the best method seems to be python_MICE_ExtraTreesRegressor, which is consistent with what was tested on other datasets available.

In [ ]:
df_RMSE = choose_best_imputation_method(X_with_Nan.iloc[:100,1000 :], path_peakTable_imputed, number_iterations=1)

In [ ]:
df_RMSE

In [ ]:
plt.figure(figsize=(5, 5))
plot_RMSE_results(df_RMSE)


<div class="alert alert-block alert-info">
We apply the extra tree Regressor estimator (mutivariate Imputation by chain equation) method to impute the missing values.
It's crashing if we have all the values, we try only with decorrelated values (twice less columns)

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
estimator_MICE = ExtraTreesRegressor(n_estimators=10, random_state=0)

In [ ]:
#print(f"- {str(estimator_MICE).split('(')[0]} :")
#intensities_python_MICE = python_MICE_imputer(X_with_Nan, estimator_MICE)
#filename = 'intensities_python_MICE_' + str(estimator_MICE).split('(')[0] + '.csv'
#intensities_python_MICE.to_csv(path_peakTable_imputed + filename, index = False, header=True)
#print(f'Imputed peak table {filename} saved in {path_peakTable_imputed}\n')

<div class="alert alert-block alert-info">
Kernel is crashing at every attempt, we try the K-nearest neighbors

In [ ]:
intensities_KNN = KNN_imputer(X_with_Nan, by='samples')

<div class="alert alert-block alert-info">
This function changes indexes : we didn't have patients 276 and 277 because we removed them, and here the indexes are reiniytialized from 0 to 376.

### We can check that there is no more missing values in the imputed peak tables

In [ ]:
plot_infos_missing_values(intensities_KNN, na_values=0)

# 4 - Transformations

In [ ]:
from normalisation_scaling_functions import *

<div class="alert alert-block alert-info">
We have a look at the distributions before applying transformations

In [ ]:
X3 = intensities_KNN.copy()

In [ ]:
plot_first_distributions_metabolites(X3)

<div class="alert alert-block alert-info">
Among the first metabolites, some are clearly not normally distributed 

In [ ]:
boxplot_first_distributions_metabolites(X3, n=100)

<div class="alert alert-block alert-info">
We can clearly observe that features are not in the same range of values, and that there are many outliers.

## a - log transformation

In [ ]:
X_log = normPeakTable(X3, 'log10', based='metabolite')
X_log

In [ ]:
boxplot_first_distributions_metabolites(X_log, n=100)

## b - Standard scale intensities

In [ ]:
X_log_std = normPeakTable(X_log, 'autoscaling', based='metabolites')

In [ ]:
boxplot_first_distributions_metabolites(X_log_std, n=100)

<div class="alert alert-block alert-info">
The features' ranges make the comparision possible.

In [ ]:
metadata2 = metadata.drop(axis = 0, index = [276, 277])

In [ ]:
X_log_std.index = metadata2.index
X_log_std.drop(axis = 1, index = 0)
X_log_std_meta = pd.concat([metadata2, X_log_std], axis=1, join = 'outer' )

In [ ]:
filename = 'intensities_decorrel_log_std.csv'
X_log_std_meta.to_csv(os.path.join(path_peakTable_imputed, filename), index = True, header=True)
print(f'Imputed peak table {filename} saved in {path_peakTable_imputed}\n')

<div class="alert alert-block alert-info">
In case you want to open the decorrelated values, uncomment the following lines

In [ ]:
peakTable_missing_log = pd.read_csv("/home/jovyan/work/data/synthetic_dataset/imputed_peak_tables/intensities_log_std.csv")
peakTable_missing_log.index = metadata2.index
peakTable_missing_log = peakTable_missing_log.drop(columns = 'Unnamed: 0', axis = 1)

In [ ]:
#peakTable_missing_log = pd.read_csv("/home/jovyan/work/data/synthetic_dataset/imputed_peak_tables/intensities_decorrel_log_std.csv")
#peakTable_missing_log.index = metadata2.index
#peakTable_missing_log = peakTable_missing_log.drop(columns = 'Unnamed: 0', axis = 1)

In [ ]:
peakTable_missing_log.index = metadata2.index